In [1]:
%matplotlib ipympl
from QDP import qdp
import numpy as np
import matplotlib.pyplot as plt
import os.path
dp = '/home/ebert/jupyter-notebooks/FNode-data'

In [7]:
exp_name = ''
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
roi_map = ['cs', 'rb']
cs_cuts = [[150],[3]]
rb_cuts = [[98],[3]]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
retention = q.apply_thresholds(exclude_rois=[2])
tbl_str = "ivar:\t{0}\tretention:{1} +- {2}"
entries = np.product(retention['retention'].shape[:-1])
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention['retention'].reshape((entries,-1))[i],
        retention['error'].reshape((entries,-1))[i]
    ))
# q.save_experiment_data()
q.experiments[0]['variable_list']
try:
    xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']
except IndexError:
    print("Only one iteration")
    xlab = ''

fig, ax = plt.subplots()
for e in range(len(retention['ivar'])):
    for r in range(2):
        ax.errorbar(
            retention['ivar'][e, :, r], retention['retention'][e, :, r],
            yerr=retention['error'][e, :, r],
            fmt=['bo','rx'][r]
        )
ax.set_ylim(0, 1.01)
ax.set_xlabel(xlab)
ax.set_ylabel('F=4(2) pop.')

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'retention_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

data at: 2018_05_09/2018_05_09_12_47_51_collisionak-retention-24ms/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'iterator']
no loading observed
no loading observed
no loading observed
no loading observed
ivar:	0.0	retention:[ 0.96052632  0.85641026         nan] +- [ 0.01219328  0.01219328  0.01219328]
ivar:	1.0	retention:[ 0.95148248  0.85454545         inf] +- [ 0.01154131  0.01154131  0.01154131]
ivar:	2.0	retention:[ 0.97039474  0.88693098         nan] +- [ 0.01020561  0.01020561  0.01020561]
ivar:	3.0	retention:[ 0.9443038   0.90086207         inf] +- [ 0.00952666  0.00952666  0.00952666]
ivar:	4.0	retention:[ 0.94524496  0.84745763         inf] +- [ 0.01391808  0.01391808  0.01391808]


FigureCanvasNbAgg()

In [8]:
dual_data = np.zeros((2, len(q.experiments[0]['iterations'].keys())))
indv_data = np.zeros((2, len(q.experiments[0]['iterations'].keys())))
dual_err = np.zeros((2, len(q.experiments[0]['iterations'].keys())))
indv_err = np.zeros((2, len(q.experiments[0]['iterations'].keys())))


for i in q.experiments[0]['iterations'].keys():
    results = q.generate_thresholds(itr=i)
    itr_cuts = {
        0: q.get_thresholds()[0],
        1: q.get_thresholds()[1]
    }
    print(itr_cuts)
    q.apply_thresholds(exclude_rois=[2])
    qd = q.experiments[0]['iterations'][i]['quantized_data']
    dual_load = np.logical_and(qd[:,0,0], qd[:,0,1])
    print(np.sum(dual_load))
    cs_only = np.logical_and(qd[:,0,0], np.logical_not(qd[:,0,1]))
    rb_only = np.logical_and(np.logical_not(qd[:,0,0]), qd[:,0,1])
    dual_ret =  np.mean(qd[dual_load,1], axis=0)[:2]
    cs_ret = np.mean(qd[cs_only,1], axis=0)[:2]
    rb_ret = np.mean(qd[rb_only,1], axis=0)[:2]
    
    dual_data[:, i] = dual_ret
    indv_data[0, i] = cs_ret[0]
    indv_data[1, i] = rb_ret[1]

    n_dual = sum(dual_load)
    n_cs = sum(cs_only)
    n_rb = sum(rb_only)

    def binomial_err(n, p):
        return np.sqrt(p*(1-p)/n)

    dual_err[:, i] = binomial_err(n_dual, np.mean(qd[dual_load,1], axis=0))[:2]
    cs_err = binomial_err(n_cs, np.mean(qd[cs_only,1], axis=0))[0]
    rb_err = binomial_err(n_rb, np.mean(qd[rb_only,1], axis=0))[1]
    indv_err[0, i] = cs_err
    indv_err[1, i] = rb_err

/usr/local/lib/python2.7/dist-packages/scipy/optimize/minpack.py:785: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  1.33200123e-18   0.00000000e+00   0.00000000e+00]`
{0: [[143], [3]], 1: [[80], [3]]}
no loading observed
no loading observed
no loading observed
143


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in sqrt


On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  1.33200123e-18   0.00000000e+00   0.00000000e+00]`
On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  1.33200123e-18   0.00000000e+00   0.00000000e+00]`
{0: [[121], [2]], 1: [[81], [3]]}
no loading observed
no loading observed
no loading observed
no loading observed
158
On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  1.33200123e-18   0.00000000e+00   0.00000000e+00]`
On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  1.33200123e-18   0.00000000e+00   0.00000000e+00]`
{0: [[129], [3]], 1: [[84], [3]]}
no loading observed
no loading observed
no loading observed
no loading observed
130
On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  1.33200123e-18   0.00000000e+00   0.0000

In [9]:
fig, ax = plt.subplots(ncols=2)
for r in range(2):
    ax[r].errorbar(
        np.arange(dual_data.shape[1]), dual_data[r],
        yerr=dual_err[r],
        fmt=['bo','rx'][r],
        label=['cs(rb)', 'rb(cs)'][r]
    )
    ax[r].errorbar(
        0.2+np.arange(dual_data.shape[1]), indv_data[r],
        yerr=indv_err[r],
        fmt=['cs','yd'][r],
        label=['cs(!rb)', 'rb(!cs)'][r]
    )
    # plot weighted means
    indv_wmean = np.dot(indv_data[r], np.power(indv_err[r], -2))/np.sum(np.power(indv_err[r], -2))
    indv_stderr = np.sqrt(np.sum(np.power(indv_err, 2)))/dual_data.shape[1]
    dual_wmean = np.dot(dual_data[r], np.power(dual_err[r], -2))/np.sum(np.power(dual_err[r], -2))
    dual_stderr = np.sqrt(np.sum(np.power(dual_err, 2)))/dual_data.shape[1]
    ax[r].axhline(indv_wmean, color='cy'[r], alpha=0.5, lw=3)
    ax[r].axhline(dual_wmean, color='br'[r], alpha=0.5, lw=3)
    # plot uncertainty of the mean
    ax[r].fill_between([0, dual_data.shape[1]], indv_wmean-indv_stderr, indv_wmean+indv_stderr, facecolor='cy'[r], alpha=0.2)
    ax[r].fill_between([0, dual_data.shape[1]], dual_wmean-dual_stderr, dual_wmean+dual_stderr, facecolor='br'[r], alpha=0.2)
    
    ax[r].set_xlabel(xlab)
    ax[r].legend(bbox_to_anchor=(0.95, 0.15), loc=1, borderaxespad=0.)
    ax[r].set_ylim(0.75, 1)
ax[0].set_ylabel('Retention')

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'dual_retention_stability.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()